In [1]:
%%configure
{"conf": {
    "spark.app.name": "Hteam_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7201,application_1618324153128_6930,pyspark,idle,Link,Link,None,
7218,application_1618324153128_6949,pyspark,busy,Link,Link,None,
7221,application_1618324153128_6952,pyspark,idle,Link,Link,None,
7223,application_1618324153128_6954,pyspark,idle,Link,Link,None,
7227,application_1618324153128_6958,pyspark,idle,Link,Link,None,


# Project for Team H - Robust Journey Planning

In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7228,application_1618324153128_6959,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']

username

'dwu'

In [4]:
#put the user name to be the same as above
username = 'dwu'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
%%local
import copy
from hdfs3 import HDFileSystem
import pandas as pd
import numpy as np
from ipywidgets import widgets
from IPython.display import display
import folium

username = 'dwu'
hdfs = HDFileSystem(host='hdfs://iccluster040.iccluster.epfl.ch', port=8020, user='ebouille') # impersonate ebouille to read the file

## 1. Prepare data

### Read data 

Read data from the timetable files

In [7]:
stop_times = spark.read.csv("/data/sbb/csv/timetable/stop_times/2019/05/07/stop_times.csv", header=True)
#stop_times.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
trips = spark.read.csv("/data/sbb/csv/timetable/trips/2019/05/07/trips.csv", header=True)
#trips.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
routes = spark.read.csv("/data/sbb/csv/timetable/routes/2019/05/07/routes.csv", header=True)
#routes.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
calendar = spark.read.csv("/data/sbb/csv/timetable/calendar/2019/05/07/calendar.csv", header=True)
#calendar.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
stops = spark.read.orc("/data/sbb/orc/geostops/")
#stops.show(3)
stops.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

39027

### Clean data

#### 1.1 Only consider stops within `ZurichHB` in 15 km 

The way of compute distance from lat and lon is from : https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula

In [12]:
from math import cos, asin, sqrt, pi, ceil, floor

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType, IntegerType, BooleanType, LongType


lat_zur = 47.378177
lon_zur = 8.540192


@F.udf(returnType=BooleanType())
def dis_filter(lat, lon):
    dis = distance(lat,lon,lat_zur,lon_zur)
    return dis < 15

#udf_func = F.udf(dis_filter, returnType=BooleanType())

stops_filt = stops.filter(dis_filter(stops.stop_lat, stops.stop_lon))
stops_filt.show(3)
stops_filt.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+-------------+--------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-------+--------------------+----------------+----------------+-------------+--------------+
|8557033|Oberhasli, Industrie|47.4592670391304|8.49001368400678|         null|              |
|8573711|   Zürich, Sädlenweg|47.3677549978138|8.48748043490521|         null|              |
|8591828|    Ebmatingen, Dorf|47.3513920447801|8.64100251698261|         null|              |
+-------+--------------------+----------------+----------------+-------------+--------------+
only showing top 3 rows

1950

Save the `stop_ids` to local so that we could use it for route planning algorithm later

In [14]:
stops_ids = stops_filt.select(['stop_id'])
stops_filt
stops_ids.write.mode("overwrite").parquet('/user/{0}/file/stops_ids/'.format(username))
stops_filt.write.mode("overwrite").parquet('/user/{0}/file/stops_filt/'.format(username))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 1.2 Only consider route in business day

We only consider `service` that are avaliable in all business days, that is avaliable from Monday to Friday

In [15]:
calendar_filt = calendar.filter( 
    (calendar.monday == True) & 
    (calendar.tuesday == True) &
    (calendar.wednesday == True) & 
    (calendar.thursday == True)& 
    (calendar.friday == True)&
    (calendar.saturday == False)&
    (calendar.sunday == False) )
# calendar_filt.show(3)
# calendar_filt.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
trips_filt = trips.join(calendar_filt, 'service_id', 'inner')\
.select('service_id', 'route_id', 'trip_id','direction_id')
# trips_filt.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
trips_filt = trips_filt.join(routes, 'route_id', 'inner')\
.select(['service_id', 'route_id', 'trip_id','route_desc','route_type','direction_id'])
# trips_filt.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 1.3 only consider business hour: 7 - 18

In [18]:
stop_times_filt = stop_times.join(trips_filt, 'trip_id', 'inner').drop(*['pickup_type', 'drop_off_type'])
#stop_times_filt.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#hour = F.udf(lambda x: x.hour, IntegerType())
#stop_times_filt = stop_times_filt.withColumn('arrival_time', F.unix_timestamp(stop_times_filt.arrival_time, 'HH:mm:ss'))
#stop_times_filt = stop_times_filt.withColumn('departure_time', F.unix_timestamp(stop_times_filt.departure_time, 'HH:mm:ss'))                                            

stop_times_filt = stop_times_filt.filter(
    (F.hour(stop_times_filt.departure_time) >= 7 )&
    (F.hour(stop_times_filt.departure_time) <= 18 )&
    (F.hour(stop_times_filt.arrival_time) >= 7 )&
    (F.hour(stop_times_filt.arrival_time) <= 18 )
)


# stop_times_filt.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 1.4 build a timetable 

We build a timetable for the route planning

In [20]:
timetable = stop_times_filt.join(stops_filt,'stop_id','inner')

timetable.cache()
# timetable.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[stop_id: string, trip_id: string, arrival_time: string, departure_time: string, stop_sequence: string, service_id: string, route_id: string, route_desc: string, route_type: string, direction_id: string, stop_name: string, stop_lat: double, stop_lon: double, location_type: int, parent_station: string]

In [21]:
timetable = timetable.withColumn('arrival_time', F.unix_timestamp(timetable.arrival_time, 'HH:mm:ss'))
timetable = timetable.withColumn('departure_time', F.unix_timestamp(timetable.departure_time, 'HH:mm:ss'))
# timetable.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
timetable =  timetable.withColumn('stop_sequence',timetable.stop_sequence.cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We save the timetable to local to run the route planning algorithm in local

In [23]:
timetable.write.mode("overwrite").parquet('/user/{0}/file/timetable/'.format(username))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 1.5 Build walk path

We build the walk path between each two stops here

In [24]:
stop_ids_lat_lon = {}
for row in stops_filt.collect():
    stop_ids_lat_lon[row['stop_id']] = (row['stop_lat'],row['stop_lon'])
stops_filt.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+-------------+--------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-------+--------------------+----------------+----------------+-------------+--------------+
|8557033|Oberhasli, Industrie|47.4592670391304|8.49001368400678|         null|              |
|8573711|   Zürich, Sädlenweg|47.3677549978138|8.48748043490521|         null|              |
|8591828|    Ebmatingen, Dorf|47.3513920447801|8.64100251698261|         null|              |
+-------+--------------------+----------------+----------------+-------------+--------------+
only showing top 3 rows

In [25]:
stops_filt2 = stops_filt.select(['stop_id','stop_name','stop_lat','stop_lon'])
stops_filt2.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|
+-------+--------------------+----------------+----------------+
|8557033|Oberhasli, Industrie|47.4592670391304|8.49001368400678|
|8573711|   Zürich, Sädlenweg|47.3677549978138|8.48748043490521|
|8591828|    Ebmatingen, Dorf|47.3513920447801|8.64100251698261|
+-------+--------------------+----------------+----------------+
only showing top 3 rows

In [26]:
stops_filt3 = stops_filt.select(['stop_id','stop_name','stop_lat','stop_lon'])\
.withColumnRenamed('stop_id','stop_id2' )\
.withColumnRenamed('stop_name','stop_name2' )\
.withColumnRenamed('stop_lat','stop_lat2' )\
.withColumnRenamed('stop_lon','stop_lon2' )
stops_filt3.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----------------+----------------+
|stop_id2|          stop_name2|       stop_lat2|       stop_lon2|
+--------+--------------------+----------------+----------------+
| 8557033|Oberhasli, Industrie|47.4592670391304|8.49001368400678|
| 8573711|   Zürich, Sädlenweg|47.3677549978138|8.48748043490521|
| 8591828|    Ebmatingen, Dorf|47.3513920447801|8.64100251698261|
+--------+--------------------+----------------+----------------+
only showing top 3 rows

The walk path between two stops need to satisfy two conditions:
- The distance between two stops need to be in 500m
- The minimum walking time is 2 min

In [27]:
stops_walk = stops_filt3.crossJoin(stops_filt2)

@F.udf(returnType=BooleanType())
def dis_filter(lat1, lon1,lat,lon):
    dis = distance(lat1,lon1,lat,lon)
    return dis < 0.5

@F.udf(returnType=LongType())
def walk_time_calculator(lat1, lon1,lat,lon):
    dis = distance(lat1,lon1,lat,lon)
    t = long(((dis*1000)/50)*60)
    return t 

stops_walk = stops_walk\
.filter(
    (dis_filter(stops_walk.stop_lat,stops_walk.stop_lon,stops_walk.stop_lat2,stops_walk.stop_lon2))&
    (stops_walk.stop_id != stops_walk.stop_id2)
)\
.withColumn('walk_time',\
               walk_time_calculator(stops_walk.stop_lat,stops_walk.stop_lon,stops_walk.stop_lat2,stops_walk.stop_lon2))
# stops_walk.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
stops_walk.write.mode("overwrite").parquet('/user/{0}/file/stops_walk/'.format(username))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#stops_walk.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2 Predict modeling

In this part, we build our predictive model to analysis the propability distribution of delay time for each stops based on the `istdaten` data

### 2.1. Prepare data

We filter and clean the data to incorperate with the `timetable` data.

We clean the data based on the same critieria as the the `timetable` data:
- We only consider data in weekdays
- We only consider data in business hours (7am - 18pm)
- We only consider stops that is in 15 km of `Zurich`

In [30]:
from pyspark.sql import functions as F
import numpy as np

# Load actual data
time_table = spark.read.orc("/data/sbb/orc/istdaten/").select("bpuic","produkt_id","ankunftszeit","an_prognose",
                                                              "an_prognose_status","faellt_aus_tf","abfahrtszeit")
# Rename columns for convenience
time_table=time_table.withColumnRenamed("bpuic","stop_id")
time_table=time_table.withColumnRenamed("produkt_id","transport_type")
time_table=time_table.withColumnRenamed("ankunftszeit","schedule")
time_table=time_table.withColumnRenamed("an_prognose","actual")
time_table=time_table.withColumnRenamed("an_prognose_status","arrival_status")
time_table=time_table.withColumnRenamed("faellt_aus_tf","journey_fail")
time_table=time_table.withColumnRenamed("abfahrtszeit","schedule_departure")
time_table.show(3)
# Convert format
time_table=time_table.withColumn("transport_type", F.lower(time_table.transport_type))
time_table=time_table.withColumn("actual", F.unix_timestamp(time_table.actual, 'dd.MM.yyy HH:mm:ss'))
time_table=time_table.withColumn("schedule", F.unix_timestamp(time_table.schedule, 'dd.MM.yyy HH:mm'))
time_table=time_table.withColumn("schedule_departure", F.unix_timestamp(time_table.schedule_departure, 'dd.MM.yyy HH:mm'))
time_table=time_table.withColumn("hour", F.hour(time_table.schedule.cast("timestamp")))
# time_table.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+----------------+-------------------+--------------+------------+------------------+
|stop_id|transport_type|        schedule|             actual|arrival_status|journey_fail|schedule_departure|
+-------+--------------+----------------+-------------------+--------------+------------+------------------+
|8580472|           Bus|02.02.2020 22:55|02.02.2020 22:57:31|      PROGNOSE|       false|  02.02.2020 22:55|
|8580521|           Bus|02.02.2020 22:56|02.02.2020 22:58:19|      PROGNOSE|       false|  02.02.2020 22:56|
|8573656|           Bus|02.02.2020 22:56|02.02.2020 22:59:07|      PROGNOSE|       false|  02.02.2020 22:57|
+-------+--------------+----------------+-------------------+--------------+------------+------------------+
only showing top 3 rows

In [31]:
# Filter
# only consider actual arrival time
time_table=time_table.filter(time_table.arrival_status.isin(['GESCHAETZT', 'PROGNOSE', 'REAL'])).drop("arrival_status")
# filter out failed trip
time_table=time_table.filter(time_table.journey_fail!=True).drop("journey_fail")
# only consider data in 7:00-18:00
time_table=time_table.filter((time_table.hour>=7)&(time_table.hour<=18)) 
time_table=time_table.filter((F.hour(time_table.schedule_departure.cast("timestamp"))>=7)&
                             (F.hour(time_table.schedule_departure.cast("timestamp"))<=18))
time_table=time_table.filter((F.hour(time_table.schedule_departure.cast("timestamp"))>=7)&
                             (F.hour(time_table.schedule_departure.cast("timestamp"))<=18))
# only consider data in weekdays
time_table=time_table.filter((F.dayofweek(time_table.schedule.cast("timestamp"))>1)&
                             (F.dayofweek(time_table.schedule.cast("timestamp"))<7))
time_table=time_table.filter((F.dayofweek(time_table.schedule_departure.cast("timestamp"))>1)&
                             (F.dayofweek(time_table.schedule_departure.cast("timestamp"))<7))
time_table=time_table.drop("schedule_departure")
# only consider stops within ZurichHB in 15 km
valid_stops = stops_filt.select('stop_id').distinct()
time_table=time_table.join(valid_stops, 'stop_id', 'inner')

time_table.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+----------+----------+----+
|stop_id|transport_type|  schedule|    actual|hour|
+-------+--------------+----------+----------+----+
|8582005|           bus|1570721160|1570721162|  17|
|8580620|           bus|1570721460|1570721505|  17|
|8582005|           bus|1570721580|1570721700|  17|
|8580617|           bus|1570721640|1570721748|  17|
|8580617|           bus|1570686960|1570687003|   7|
|8582005|           bus|1570686960|1570687051|   7|
|8580620|           bus|1570687260|1570687309|   8|
|8582005|           bus|1570687380|1570687417|   8|
|8580617|           bus|1570687440|1570687477|   8|
|8580617|           bus|1570685160|1570685160|   7|
|8582005|           bus|1570685160|1570685208|   7|
|8580620|           bus|1570685460|1570685516|   7|
|8582005|           bus|1570685580|1570685623|   7|
|8580617|           bus|1570685640|1570685659|   7|
|8580617|           bus|1570692360|1570692360|   9|
|8582005|           bus|1570692360|1570692408|   9|
|8580620|   

### 2.2 calculate the probability for delays
We calculate the success probability for each stop at different hours with the time tolerance for arriving at the next stop.

#### The model of delay:

We model our delay at each stops as follows:
- We model our delay at each stops $s$ as are random variable $X$, which is a function of hour $h$, transport type $t$, that is $X = X(s,h,t)$. The analysis what factors are the delay random variables related tois shown in the `predict_analysis.ipynb`
- Given the stop_id $s$, hour $h$, transport type $t$, we need to calculate the  probability that a trip's delay is less than time tolerance $d$, that is :
$$Pr(X<d) $$
- Given the stop_id $s$, hour $h$, transport type $t$, nn order to estimate the above probability,we assume that the delay different trips are independent, that is the delay for each trips are i.i.d samples $X_1,X_2,..,X_n$ of $X(s,h,t)$. Then we estimate the probability $Pr(X<d) $ as follows:
$$Pr(X<d) = \frac{1}{n} 1\{X<d\} = freq(X<d)$$
- More percisely, stop_id $s$, hour $h$, transport type $t$, and time tolerance $d$, we calculate the frequence that the delay of the arrival time of the trips is less than the time tolerance, and use the frequence as the probability
- By law of large number, we know that $freq(X<d)$ is an unbiased estimator of the probability $Pr(X<d)$



In [ ]:
# Calculate delay distribution
delay_table=time_table.withColumn('delay', F.ceil((time_table.actual-time_table.schedule)/60)).drop("schedule", "actual")
# delay_table.show(3)
delay_table=delay_table.groupBy(["stop_id","transport_type", "hour", "delay"]).count().withColumnRenamed("count","delay_count")
#delay_table.cache()
delay_table.show(3)

In [ ]:
delay_table.write.mode("overwrite").parquet('/user/{0}/file/delay_table/'.format(username))

In [32]:
%%local
files = hdfs.glob('/user/{0}/file/delay_table/*.parquet'.format(username))
delay_table = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        delay_table = delay_table.append(pd.read_parquet(f))



In [33]:
%%local
delay_table.head(3)

Output()

In [34]:
def get_on_time_prob(stop_id, transport_type, hour, time_tolerance):
    """ Get probabiltiy P(wait_time<=time_tolerance)
    Args:
        stop_id (int or str):
        transport_type (str):
        hour (int): hour of a day
        time_tolerance (int): delay in minutes
    Outputs:
        prob (float):
    """
    
    delay_dist = np.array(delay_table.filter((delay_table.stop_id==stop_id)&
                                             (delay_table.transport_type==transport_type.lower())&
                                             (delay_table.hour==hour)).select("delay", "delay_count").collect())
    if delay_dist.size==0:
        return 0
    
    success=0
    for minute, count in delay_dist:
        if minute<=time_tolerance:
            success+=count
    return float(success)/(delay_dist[:,1].sum())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The local version of get probability 

In [40]:
%%local
def get_on_time_prob(stop_id, transport_type, hour, time_tolerance):
    """ Get probabiltiy P(wait_time<=time_tolerance)
    Args:
        stop_id (str):
        transport_type (str):
        hour (int): hour of a day
        time_tolerance (int): delay in minutes
    Outputs:
        prob (float):
    """
    
    delay_dist = np.array(delay_table[(delay_table.stop_id==stop_id)&
                                             (delay_table.transport_type==transport_type.lower())&
                                             (delay_table.hour==hour)][["delay", "delay_count"]].values)
    if delay_dist.size==0:
        return 0
    
    success=0
    for minute, count in delay_dist:
        if minute<=time_tolerance:
            success+=count
    return float(success)/(delay_dist[:,1].sum())

In [41]:
%%local
# one example
print(get_on_time_prob(stop_id='8503308', transport_type='Zug', hour=9, time_tolerance=5))

0.978000647039793


In [42]:
%%local
print(get_on_time_prob(stop_id='8503308', transport_type='Zug', hour=7, time_tolerance=5))

0.9798897178073305


## 3 Algorithm for route planning

### 3.1 Load the data that we store in loacal before

We get run the algorithms on local for faster speed

In [44]:
%%local
import copy
from hdfs3 import HDFileSystem
username = 'dwu'
hdfs = HDFileSystem(host='hdfs://iccluster040.iccluster.epfl.ch', port=8020, user='ebouille') # impersonate ebouille to read the file
import pandas as pd
files = hdfs.glob('/user/{0}/file/timetable/*.parquet'.format(username))
timetable = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        timetable = timetable.append(pd.read_parquet(f))
        
files = hdfs.glob('/user/{0}/file/stops_walk/*.parquet'.format(username))
stops_walk = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        stops_walk = stops_walk.append(pd.read_parquet(f))
        
        
files = hdfs.glob('/user/{0}/file/stops_ids/*.parquet'.format(username))
stops_ids = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        stops_ids = stops_ids.append(pd.read_parquet(f))

stops_ids = stops_ids['stop_id'].unique().tolist()

files = hdfs.glob('/user/{0}/file/stops_filt/*.parquet'.format(username))
stops_filt = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        stops_filt = stops_filt.append(pd.read_parquet(f))

In [45]:
%%local
stops_filt.head(3)
stops_ids[340]

'8503315:0:2'

### 3.2 Implement of the algorithm
We implement 2 algorithm:
1. The RAPTOR algorithm from this paper:https://www.microsoft.com/en-us/research/wp-content/uploads/2012/01/raptor_alenex.pdf 
2. A modified version of RAPTOR algorithm, so that it can find the optimal path with the success probability larger than Q

In [46]:
%%local
import numpy as np
from ipywidgets import widgets
from IPython.display import display
import folium

In [47]:
%%local
walking_table = stops_walk[['stop_id','stop_id2','walk_time']]

#### The original implementation of the RAPTOR algorithm


The basic idea of the algorithm is board first search and target pruning. 

The algorithm is shown as follows:
- Input: `source_stop`, `destination_stop`, `begin_time`, `time_limit`, `max_trans`
- Output: the optimal route in  `time_limit`, and with at most `max_trans` transfers (walk path also consider as a trasfer)
- 1. Intialize the a `result_time_table[p][i]`, which represent the optimal arrival time at stop `p` with at most `i` tranfers, we set `result_time_table[source_stop][0]= begin_time`, and mark `source_stop`
- 2. At each round `i`, for all marked stops `p`, we find all trips that is valid w.r.t `result_time_table[p][i]`, that is a trip whose `depature_time > result_time_table[p][i]`
- 3. For each trip `t` we found above, we traverse all the subsequent stops `p_t` in this trip, and if it is a better trip for `p_t`(that is `result_time_table[p_t][i] > arrival_time[p_t]`), we mark `p_t`.
- 4. We run until there are no stops that is marked

In [48]:
%%local
def find_walking(stop_id, time):
    walking_next_stop_list = walking_table[walking_table['stop_id']== stop_id][['stop_id2']].values.tolist()
    walking_next_time_list = (walking_table[walking_table['stop_id']== stop_id][['walk_time']].values + time).tolist()
    return walking_next_stop_list, walking_next_time_list
    
    
def find_trip(stop_id, time, time_table):
    return time_table[np.logical_and(time_table['stop_id']== stop_id, \
                                     time_table['departure_time']>= time)][["trip_id"]].values.tolist()


def getBestRoutes(source_stop, destination_stop, begin_time, time_limit, max_trans):
    time_table = timetable[np.logical_and(timetable['departure_time']>= begin_time, timetable['arrival_time']<= (begin_time + time_limit))]
    
    result_time_table = np.zeros([len(stops_ids), max_trans], dtype = int, order = 'C')
    result_time_table = result_time_table - 1
    result_path_table = np.zeros([len(stops_ids), max_trans], dtype = int, order = 'C')
    result_path_table = result_time_table - 1
    source_stop_index = stops_ids.index(source_stop)
    result_time_table[source_stop_index] = begin_time
    result_path_table[source_stop_index] = -2
    
    result_trip_info_table = np.zeros([len(stops_ids), max_trans], dtype = int, order = 'C')
    result_trip_info_table = result_trip_info_table - 1
    
    trips_list = timetable[['trip_id']].drop_duplicates().values.tolist()
    trips_list = [r[0] for r in trips_list]
    
    
    

    stop_list_next_iter = []
    stop_list_next_iter.append(source_stop)
    stop_list = []
    time_list_next_iter = []
    time_list_next_iter.append(begin_time)
    time_list = []
    
    
    for i in range(max_trans):
        for j in range(len(stop_list_next_iter)):
        
            now_index = stops_ids.index(stop_list_next_iter[j])
            stop_result, time_result= find_walking(stop_list_next_iter[j], time_list_next_iter[j])
            
            # find all walk path
            for k in range(len(stop_result)):
                stop_index = stops_ids.index(stop_result[k][0])
                if result_time_table[stop_index][0] == -1:
                    result_time_table[stop_index] = time_result[k][0]
                    result_path_table[stop_index] = now_index
                    result_trip_info_table[stop_index] = -100
                    stop_list.append(stop_result[k][0])
                elif result_time_table[stop_index][i] > time_result[k][0]:
                    for l in range(max_trans - i):
                        result_time_table[stop_index][i+l] = time_result[k][0]
                        result_path_table[stop_index][i+l] = now_index
                        result_trip_info_table[stop_index][i+l] = -100 #represent walk
                        
                    stop_list.append(stop_result[k][0])
                    
            # find all valid trips        
            trip_result = find_trip(stop_list_next_iter[j], time_list_next_iter[j], time_table) 
            
            for s in range(len(trip_result)):
                    
                trip_time_table = time_table[time_table['trip_id']== trip_result[s][0]]
                stop_arrival_time = trip_time_table[trip_time_table['stop_id']== stop_list_next_iter[j]][["arrival_time"]].values.tolist()[0][0]
                stop_result = trip_time_table[trip_time_table['arrival_time']> stop_arrival_time][["stop_id"]].values.tolist()
                time_result = trip_time_table[trip_time_table['arrival_time']> stop_arrival_time][["arrival_time"]].values.tolist()
                
                
                
                for k in range(len(stop_result)):
                    stop_index = stops_ids.index(stop_result[k][0])
                    if result_time_table[stop_index][0] == -1:
                        result_time_table[stop_index] = time_result[k][0]
                        result_path_table[stop_index] = now_index
                        result_trip_info_table[stop_index] = trips_list.index(trip_result[s][0])
                        stop_list.append(stop_result[k][0])
                    elif result_time_table[stop_index][i] > time_result[k][0]:
                        for l in range(max_trans - i):
                            result_time_table[stop_index][i+l] = time_result[k][0]
                            result_path_table[stop_index][i+l] = now_index 
                            #result_path_table[stop_index] = now_index 
                            result_trip_info_table[stop_index][i+l] = trips_list.index(trip_result[s][0])
                        stop_list.append(stop_result[k][0])
        stop_list_next_iter = list(set(stop_list))
        stop_list = []
        for j in range(len(stop_list_next_iter)):
            stop_index = stops_ids.index(stop_list_next_iter[j])
            time_list.append(result_time_table[stop_index][i])
        time_list_next_iter = time_list
        time_list = []
    
    final_result = []
    final_time = []
    final_trips = []
    now_index = stops_ids.index(destination_stop)
    i = max_trans - 1
    #while(True):
    while(True):
        final_result.append(now_index)
        final_time.append(result_time_table[now_index][i])
        if (result_time_table[now_index][i] == -1) and (now_index != stops_ids.index(source_stop)):
            print('no path is found')
            break;
            
        trip_index =  result_trip_info_table[now_index][i]
        if trip_index == -100:
            final_trips.append('walk')
        elif trip_index>-1:
            final_trips.append(trips_list[trip_index])
        else:
            final_trips.append(trip_index)
        now_index = result_path_table[now_index][i]
        i = i - 1
        if now_index == -2:
            break
    
    return final_result, final_time, final_trips

#### Modified version of RAPTOR Algorithm

Idea: The RAPTOR Algorithm is based on board first search, this is not suitable to for calculate the success probability, because in board first search, one can not see a complete route; however we can get a complete route at each round for deep first search. So in this algorithm, we change the board first search to deep first search, and the rest is same.

In [49]:
%%local
def getBestRoutes_DFS(source_stop, destination_stop, begin_time, time_limit, max_trans, min_prob):
    time_table = timetable[np.logical_and(timetable['departure_time']>= begin_time, timetable['arrival_time']<= (begin_time + time_limit))]
    
    result_time_table = np.zeros([len(stops_ids), max_trans], dtype = int, order = 'C')
    result_time_table = result_time_table - 1
    result_path_table = np.zeros([len(stops_ids), max_trans], dtype = int, order = 'C')
    result_path_table = result_time_table - 1
    source_stop_index = stops_ids.index(source_stop)
    result_time_table[source_stop_index] = begin_time
    result_path_table[source_stop_index] = -2
    
    result_prob_table = np.zeros([len(stops_ids), max_trans], dtype = int, order = 'C')
    result_prob_table = result_time_table - 1
    
    result_trip_info_table = np.zeros([len(stops_ids), max_trans], dtype = int, order = 'C')
    result_trip_info_table = result_trip_info_table - 1
    
    trips_list = timetable[['trip_id']].drop_duplicates().values.tolist()
    trips_list = [r[0] for r in trips_list]
    
    #min_prob = min_prob**(1/float(max_trans))
    #result_trip_info_table[now_index][i]
    

    #stop_list_next_iter = []
    #stop_list_next_iter.append((source_stop,0))
    
    stop_list = []
    stop_list.append((source_stop,0))
    #time_list_next_iter = []
    #time_list_next_iter.append(begin_time)
    time_list = []
    time_list.append((begin_time))
    
    
    while len(stop_list) > 0:
        
        s = stop_list.pop()
        curr_stop_id = s[0]
        curr_count = s[1]
        curr_arr_time  = time_list.pop()
        
        #print(s)
    
        if curr_count < max_trans:
            
            i = curr_count
            if curr_stop_id == source_stop and curr_count == 0:
                trans_count = 0
            else:
                trans_count = i+1
            
            now_index = stops_ids.index(curr_stop_id)
            stop_result, time_result= find_walking(curr_stop_id, curr_arr_time)
            
            # find all walk path
            for k in range(len(stop_result)):
                stop_index = stops_ids.index(stop_result[k][0])
                if result_time_table[stop_index][0] == -1:
                    result_time_table[stop_index] = time_result[k][0]
                    result_path_table[stop_index] = now_index
                    result_trip_info_table[stop_index] = -100
                    result_prob_table[stop_index] = 1
                    if (stop_result[k][0],i+1) not in stop_list:
                                stop_list.append((stop_result[k][0],trans_count))
                                time_list.append(time_result[k][0])
                elif result_time_table[stop_index][i] > time_result[k][0]:
                    for l in range(max_trans - i):
                        result_time_table[stop_index][i+l] = time_result[k][0]
                        result_path_table[stop_index][i+l] = now_index
                        result_trip_info_table[stop_index][i+l] = -100 #represent walk                       
                    if (stop_result[k][0],i+1) not in stop_list:
                                stop_list.append((stop_result[k][0],trans_count))
                                time_list.append(time_result[k][0])
                    
            # find all valid trips        
            trip_result = find_trip(curr_stop_id, curr_arr_time, time_table) 
            
            for s in range(len(trip_result)):

                trip_time_table = time_table[time_table['trip_id']== trip_result[s][0]]
                stop_arrival_time = trip_time_table[trip_time_table['stop_id']== curr_stop_id][["arrival_time"]].values.tolist()[0][0]
                stop_departure_time = trip_time_table[trip_time_table['stop_id']== curr_stop_id][["departure_time"]].values.tolist()[0][0]
                stop_result = trip_time_table[trip_time_table['arrival_time']> stop_arrival_time][["stop_id"]].values.tolist()
                time_result = trip_time_table[trip_time_table['arrival_time']> stop_arrival_time][["arrival_time"]].values.tolist()
                
                if trip_result[s][0] is not result_trip_info_table[now_index][i]:
                    # which means we have a transfer here
                    trip_id = trip_result[s][0]
                    transport_type = trip_time_table[trip_time_table['stop_id']== curr_stop_id][["route_desc"]].values.tolist()[0][0]
                    hour = int(stop_arrival_time/3600)
                    time_tolerance = stop_departure_time - result_time_table[now_index][i]
                    prob = get_on_time_prob(stop_id=curr_stop_id, transport_type=transport_type, hour=hour, time_tolerance=time_tolerance)
                    if prob*result_prob_table[now_index][i] < min_prob:
                        continue
                    else:
                        result_prob_table[now_index][i] = prob*result_prob_table[now_index][i]
                
                for k in range(len(stop_result)):
                    
                    stop_index = stops_ids.index(stop_result[k][0])
                    if result_time_table[stop_index][0] == -1:
                        result_time_table[stop_index] = time_result[k][0]
                        result_path_table[stop_index] = now_index
                        result_trip_info_table[stop_index] = trips_list.index(trip_result[s][0])
                        result_prob_table[stop_index] = 1
                        if (stop_result[k][0],i+1) not in stop_list:
                            stop_list.append((stop_result[k][0],trans_count))
                            time_list.append(time_result[k][0])
                    elif result_time_table[stop_index][i] > time_result[k][0]:
                        for l in range(max_trans - i):
                            result_time_table[stop_index][i+l] = time_result[k][0]
                            result_path_table[stop_index][i+l] = now_index 
                            #result_path_table[stop_index] = now_index 
                            result_trip_info_table[stop_index][i+l] = trips_list.index(trip_result[s][0])
                            if (stop_result[k][0],i+1) not in stop_list:
                                stop_list.append((stop_result[k][0],trans_count))
                                time_list.append(time_result[k][0])
        #stop_list_next_iter = list(set(stop_list))
        #stop_list = []
        #for j in range(len(stop_list_next_iter)):
        #    stop_index = stops_ids.index(stop_list_next_iter[j])
        #    time_list.append(result_time_table[stop_index][i])
        #time_list_next_iter = time_list
        #time_list = []
    
    final_result = []
    final_time = []
    final_trips = []
    now_index = stops_ids.index(destination_stop)
    i = max_trans - 1
    #while(True):
    while(True):
        final_result.append(now_index)
        final_time.append(result_time_table[now_index][i])
        if (result_time_table[now_index][i] == -1) and (now_index != stops_ids.index(source_stop)):
            print('no path is found')
            break;
            
        trip_index =  result_trip_info_table[now_index][i]
        if trip_index == -100:
            final_trips.append('walk')
        elif trip_index>-1:
            final_trips.append(trips_list[trip_index])
        else:
            final_trips.append(trip_index)
        now_index = result_path_table[now_index][i]
        i = i - 1
        if now_index == -2:
            break
    
    return final_result, final_time, final_trips

Here are some simple example for our algorithm, just to show that our algorithm works. A more conrete validation will be shown in next part.

In [50]:
%%local
final_result, final_time, final_trips= getBestRoutes('8590615','8590271', 36899, 7200, 3)
final_result, final_time, final_trips

([684, 1664, 1850, 1511],
 [41664, 41520, 39660, 36899],
 ['walk', '224.TA.26-303-j19-1.3.R', '221.TA.26-302-j19-1.5.H', -1])

In [ ]:
%%local
final_result, final_time, final_trips= getBestRoutes('8590615','8590271', 36899, 7200, 3)
final_result, final_time, final_trips

Here we set the low probability low, to make sure it works, A more concrete validation will be shown in next part

Note: This algorithm may need to run for few minutes

In [ ]:
%%local
final_result, final_time, final_trips = getBestRoutes_DFS('8590615','8590271', 36899, 7200, 3, 0.1)
final_result, final_time, final_trips

This function is to change the result in forms that could be used in later processing

In [51]:
%%local
def change_result_format(final_result,final_time,final_trips):
    final_result = final_result[::-1]
    final_time = final_time[::-1]
    final_trips = final_trips[::-1]
    
    final_departure = []
            
        
    
    path_list = []
    trip_list = []
    time_list = []
    
    for i,stop_index in enumerate(final_result):
        stop_id = stops_ids[stop_index]
        if i == 0:
            trip_id = final_trips[i+1]
            path_list.append(stop_id)
            trip_list.append(trip_id)
            if trip_id is not 'walk':
                
                stop_dep_time = timetable[(timetable['stop_id']==stop_id)&
                                        (timetable['trip_id']==trip_id)][["departure_time"]].values.tolist()[0][0]
                
                time_list.append(stop_dep_time)
                
            else:
                time_list.append( final_time[i])
            
        elif i == len(final_result)-1:
            trip_id = final_trips[i]
            path_list.append(stop_id)
            time_list.append(final_time[i])
            trip_list.append(trip_id)
            
        else:
            trip_id = final_trips[i+1]
            path_list.append(stop_id)
            path_list.append(stop_id)
            trip_list.append(trip_id)
            trip_list.append(trip_id)
            if trip_id is not 'walk':
                #print((trip_id,stop_id))
                stop_dep_time = timetable[(timetable['stop_id']==stop_id)&
                                        (timetable['trip_id']==trip_id)][["departure_time"]].values.tolist()[0][0]

                time_list.append(final_time[i])
                time_list.append(stop_dep_time)
            else: 
                time_list.append(final_time[i])
                time_list.append(final_time[i])
                
    trip_list = trip_list[:-1]
    trip_list.insert(0,trip_list[0])
        
    return path_list, time_list,trip_list
        
    

In [53]:
%%local
path_list, time_list,trip_list = change_result_format(final_result, final_time, final_trips)
path_list, time_list,trip_list

(['8590615', '8590794', '8590794', '8591891', '8591891', '8590271'],
 [38460, 39660, 40200, 41520, 41520, 41664],
 ['221.TA.26-302-j19-1.5.H',
  '221.TA.26-302-j19-1.5.H',
  '224.TA.26-303-j19-1.3.R',
  '224.TA.26-303-j19-1.3.R',
  'walk',
  'walk'])

## 4 Validation of Methods

### 4.1. Compare with Google Maps application


In our vedio, we demonstrate an example, the upper picture is the one given by google maps application, and the lower one is our recommendation. It can be told that these two are quite similar.
starting point: Geroldswil, Grindlen
destination:Spreitenbach, Dorf

### 4.2. Compute the path completion probability of our recommendation
First, we recomend a path

In [54]:
%%local
stop_id = ['8590615', '8590805', '8590805', '8591891', '8591891', '8590271']
arrival_time = [38460, 39660, 40200, 41520, 41520, 41664]
final_trip =  ['221.TA.26-302-j19-1.5.H',
  '221.TA.26-302-j19-1.5.H',
  '224.TA.26-303-j19-1.3.R',
  '224.TA.26-303-j19-1.3.R',
  'walk',
  'walk']
def path_todict(final_trips, stop_ids, arrival_times):
    path_list = []
    for trip_id, stop_id, arrival_time in zip(final_trips,stop_ids, arrival_times):
        stop_dict ={}
        stop_dict['trip_id'] = trip_id
        stop_dict['stop_id'] = stop_id
        stop_dict['arrival_time'] = arrival_time
        path_list.append(stop_dict)
    return path_list




In [55]:
%%local
def split_route(route):
    '''
    Split the recommended path into segments using the same transport type
    '''
    splitted_stop_list = []
    splitted_time_list = []
    splitted_id_list = []
    segment = None
    timeline = None
    idline = None
    for id_, stop in enumerate(route):
        if id_ == 0 or trip_id != stop['trip_id']:
            trip_id = stop['trip_id']
            if id_ != 0:
                splitted_stop_list.append(segment)
                splitted_time_list.append(timeline)
                splitted_id_list.append(idline)
            segment = [stop['stop_id']]
            timeline = [stop['arrival_time']]
            idline = [stop['trip_id']]
        else:
            segment.append(stop['stop_id'])
            timeline.append(stop['arrival_time'])
            idline.append(stop['trip_id'])
    splitted_stop_list.append(segment)
    splitted_time_list.append(timeline)
    splitted_id_list.append(idline)
    
            
    return splitted_stop_list, splitted_time_list, splitted_id_list
        
stop_list, time_list, trip_list = split_route(path_list)
print(stop_list)
print(time_list)
print(trip_list)

TypeError: string indices must be integers

In [56]:
%%local
files = hdfs.glob('/user/{0}/file/delay_table/*.parquet'.format('dwu'))
delay_table = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        delay_table = delay_table.append(pd.read_parquet(f))

In [57]:
%%local
import numpy as np
def get_on_time_prob(stop_id, transport_type, hour, time_tolerance):

    delay_dist = np.array(delay_table[(delay_table['stop_id']==stop_id)&
                                        (delay_table['transport_type']==transport_type.lower())&
                                      (delay_table['hour']==hour)][["delay","delay_count"]])
    if delay_dist.size==0:
        return 0

    success=0
    for minute, count in delay_dist:
        if minute<=time_tolerance:
            success+=count
    return float(success)/(delay_dist[:,1].sum())

Then, compute the probability with the record of SBB.

In [58]:
%%local ## working here
from math import floor
def path_completion_prob(stop_list, time_list, trip_list, stops_walk, delay_table, timetable):

    completion_prob = 1
    for i, (stop_ids, arrival_times, trip_ids) in enumerate(zip(stop_list, time_list, trip_list)):
        print("=========")
        if (i<= len(stop_list) - 2) and (trip_ids[-1] != trip_list[i+1][0]):
            walk_start_pt = stop_ids[-1]
            walk_end_pt = stop_list[i+1][0]
            condition1 = (stops_walk["stop_id2"] == walk_start_pt)
            condition2 = (stops_walk["stop_id"] ==walk_end_pt)
            condition = condition1&condition2
            sub_df = stops_walk[condition]

            try:
                if walk_start_pt == walk_end_pt:
                    walk_time = 120
                else:
                    walk_time = sub_df.iat[0,-1]
    #                 print(walk_time)

                tolerance_time = time_list[i+1][0] - arrival_times[-1] - walk_time
    #                 print(time_list[i+1][0])
    #                 print(arrival_times[-1])
    #                 print(tolerance_time)
                if trip_ids[-1] == 'walk':
                    transit_on_time_prob = 1 if tolerance_time >0 else 0
                else:
                    condition3 = timetable["stop_id"] == stop_ids[-1]
                    condition4 = timetable["trip_id"] == trip_ids[-1]
                    condition_ = condition3 & condition4
                    transport_type_sub_df = timetable[condition_]
        #                 return transport_type_sub_df
                    transport_type = transport_type_sub_df.iloc[0].at['route_desc']
        #                 print('transport_type {}'.format(transport_type))

        #                 print('made to here')
        #                 print(transport_type)
                    hour = floor(arrival_times[-1]/3600)
                    if trip_list[i+1][0] == 'walk':
                        transit_on_time_prob = 1

                    else:
                        transit_on_time_prob = get_on_time_prob(stop_ids[-1], transport_type, hour, tolerance_time)
        #                     print(stop_ids[-1])
    #                     print(transport_type)
    #                     print(hour)
    #                     print("probability: {}".format(transit_on_time_prob))
                print("probability of completing the No{}. transition: {}".format(i+1,transit_on_time_prob))
                completion_prob*=transit_on_time_prob

            except Exception as ex:
                print(ex)
                print('walking is not an option between {} and {}'.format(walk_start_pt, walk_end_pt))
                return
    print("SUMMARY  The probability of completing the entire path is: {}".format(completion_prob))

    
    

In [59]:
%%local
## Find a bad stop, which may have large delay
hour = 10
time_tolerance = 5
min_prob = 1
min_stop =''
min_transport_type = ''
for stop in stops_ids[1601:1910]:
    
    transport_type_list = timetable[timetable['stop_id']== stop][["route_desc"]].values.tolist()
    if len(transport_type_list)>0:
        transport_type = transport_type_list[0][0]
    prob = get_on_time_prob(stop_id=stop, transport_type=transport_type, hour=hour, time_tolerance=time_tolerance)
    if (prob<min_prob) and prob>0:
        min_prob = prob
        min_stop = stop
        min_transport_type = transport_type
    

In [60]:
%%local
min_stop, min_prob

('8591428', 0.8)

In [61]:
%%local
# Here is the algorithm that we do not consider the 
final_result, final_time, final_trips= getBestRoutes('8581546',stops_ids[3], 36899, 7200, 4)
final_result, final_time, final_trips

([3, 271, 142, 1438, 222],
 [41340, 40404, 40380, 37620, 36899],
 ['213.TA.26-704-j19-1.7.R',
  'walk',
  '99.TA.26-725-j19-1.5.H',
  '54.TA.26-813-j19-1.2.R',
  -1])

In [62]:
%%local
stops_ids[3], stops_ids[271],stops_ids[142],stops_ids[1438],stops_ids[222]

('8590610', '8576127:0:B', '8576127:0:A', '8573504', '8581546')

In [ ]:
%%local
path_list, time_list,trip_list = change_result_format(final_result, final_time, final_trips)
path_dict = path_todict(trip_list, path_list, time_list)
stop_list, time_list, trip_list = split_route(path_dict)
print(stop_list, time_list, trip_list)
probability = path_completion_prob(stop_list, time_list, trip_list, stops_walk, delay_table, timetable)

As it turns out, the path we recommended from 8581546 to 8590610 has a completion probability of 100%

### 2. Verify that our designed route appears in the data pool, justifying its feasibility


In [ ]:
%%local
for i,(stop_segment, timeline, idline) in enumerate(zip(stop_list, time_list, trip_list)): #to be modified
    for j,(stop, time, trip) in enumerate(zip(stop_segment, timeline, idline)):
        condition1 = timetable["stop_id"] == stop
        condition2 = timetable["arrival_time"] == time
        condition3 = timetable["trip_id"] == trip
        condition = condition1&condition2&condition3
        same_as_recommendation = timetable[condition]
        if same_as_recommendation.stop_id.count()!=0:
            print("The No.{} stop of No.{} trip in our recommendation is scheduled".format(j+1,i+1))
        elif trip == 'walk':
            print("This part is done by walking")
        else:
            print("The No.{} stop of No.{} trip in our recommendation is scheduled".format(j+1,i+1))

As shown, beside one trip that is done by walking, all the other trips are scheduled by SBB

## 5 User Interface

In this part, we implement an user interface. In the User interface, we use the algorithm without input the minimum probability that is the original RATPOR algorithm.

In [ ]:
%%local
Label_1 = widgets.Label("Source stop")
display(Label_1)
Text_1 = widgets.Text()
display(Text_1)
Label_2 = widgets.Label("Destination stop")
display(Label_2)
Text_2 = widgets.Text()
display(Text_2)
Label_3 = widgets.Label("Start Time")
display(Label_3)
Text_3 = widgets.Text()
display(Text_3)
Label_4 = widgets.Label("Max time limit")
display(Label_4)
Text_4 = widgets.Text()
display(Text_4)
Label_5 = widgets.Label("Max transfers")
display(Label_5)
Text_5 = widgets.Text()
display(Text_5)
Button = widgets.Button(description = "Start")
display(Button)


output = widgets.Output()
display(output)

def run(b):
    QD = Text_1.value
    ZD = Text_2.value
    CFSJ = int(Text_3.value)
    ZDSJ = int(Text_4.value)
    ZDHCCS = int(Text_5.value)
    with output:
        final_result, final_time, final_trips = getBestRoutes(QD, ZD, CFSJ, ZDSJ, ZDHCCS)
        print(final_result, final_time, final_trips)
        m = folium.Map(location=[47.3842, 8.53185], zoom_start=11)
        long = stops_filt[stops_filt['stop_id']== stops_ids[final_result[0]]][['stop_lon']].values.tolist()[0][0]
        lat = stops_filt[stops_filt['stop_id']== stops_ids[final_result[0]]][['stop_lat']].values.tolist()[0][0]
        folium.Marker(location=[lat, long],popup=stops_ids[final_result[0]], tooltip=final_time[0],icon=folium.Icon(color="blue", icon="info-sign"),).add_to(m)
        
        #print(final_result, final_time, final_trips)
        
        for i in range(len(final_result)-1):
            timetable_next = stops_filt[stops_filt['stop_id']== stops_ids[final_result[i]]]
            timetable_this = stops_filt[stops_filt['stop_id']== stops_ids[final_result[i+1]]]
            
            
            long_1 = timetable_next[['stop_lon']]\
            .values.tolist()[0][0]
            lat_1 = timetable_next[['stop_lat']]\
            .values.tolist()[0][0]
            long_2 = timetable_this[['stop_lon']]\
            .values.tolist()[0][0]
            lat_2 = timetable_this[['stop_lat']]\
            .values.tolist()[0][0]
            
            arrival_time_next = final_time[i]
            arrival_time_this = final_time[i+1]
            
            trip_id = final_trips[i]
            print(trip_id)
#             trip_info = {}
#             if trip_id == 'walk':
#                 trip_info = {
#                     'trip_id':'walk',
#                     'route_desc':'walk',
#                     'departure_time' :  arrival_time_this,
#                     'arrival_time': arrival_time_next
#                 }
#             else:
                #arrival_time =  timetable_next[timetable_next['trip_id']== trip_id][['arrival_time']]\
            #.values.tolist()[0][0]
#                 departure_time_this = timetable_this[timetable_this['trip_id']== trip_id][['departure_time']]\
#             .values.tolist()[0][0]
#                 departure_time_next = timetable_next[timetable_next['trip_id']== trip_id][['departure_time']]\
#             .values.tolist()[0][0]
#                 route_desc = timetable[timetable['trip_id']== trip_id][['route_desc']]\
#             .values.tolist()[0][0]
                
#             trip_info = {
#                     'trip_id':trip_id,
#                     'route_desc': route_desc,
#                     'departure_time' :  departure_time_this,
#                     'arrival_time': arrival_time_next}
            if i == len(final_result)-2:
                folium.Marker(location=[lat_2, long_2],popup=stops_ids[final_result[i+1]],\
                          tooltip=final_time[i+1],icon=folium.Icon(color="green", icon="info-sign"),).add_to(m)
            else:
                folium.Marker(location=[lat_2, long_2],popup=stops_ids[final_result[i+1]],\
                          tooltip=final_time[i+1],icon=folium.Icon(color="red", icon="info-sign"),).add_to(m)
#             folium.PolyLine(locations=[(lat_1, long_1),(lat_2, long_2)],popup=trip_info,\
#                             tooltip=trip_info, weight=2,color = 'blue').add_to(m)
            folium.PolyLine(locations=[(lat_1, long_1),(lat_2, long_2)], weight=2,color = 'blue').add_to(m)
            
        display(m)  
Button.on_click(run)


# 8573711
# 8590610
# 34000
# 3600
# 4
# 
# 8590615
# 8590619
# 36899
# 7200
# 3
# 
# 8590615
# 8590619
# 27000
# 7200
# 4
#
#8590887
#8587020
#36899
#7200
#4


In this part, we implement an user interface. In this User interface, we use the algorithm with input the minimum probability that is our modified RATPOR algorithm.

In [ ]:
%%local
Label_1 = widgets.Label("Source stop")
display(Label_1)
Text_1 = widgets.Text()
display(Text_1)
Label_2 = widgets.Label("Destination stop")
display(Label_2)
Text_2 = widgets.Text()
display(Text_2)
Label_3 = widgets.Label("Start Time")
display(Label_3)
Text_3 = widgets.Text()
display(Text_3)
Label_4 = widgets.Label("Max time limit")
display(Label_4)
Text_4 = widgets.Text()
display(Text_4)
Label_5 = widgets.Label("Max transfers")
display(Label_5)
Text_5 = widgets.Text()
display(Text_5)
Label_6 = widgets.Label("Min probability")
display(Label_6)
Text_6 = widgets.Text()
display(Text_6)
Button = widgets.Button(description = "Start")
display(Button)


output = widgets.Output()
display(output)

def run(b):
    QD = Text_1.value
    ZD = Text_2.value
    CFSJ = int(Text_3.value)
    ZDSJ = int(Text_4.value)
    ZDHCCS = int(Text_5.value)
    min_prob = int(Text_6.value)
    with output:
        final_result, final_time, final_trips = getBestRoutes_DFS(QD, ZD, CFSJ, ZDSJ, ZDHCCS,min_prob)
        print(final_result, final_time, final_trips)
        m = folium.Map(location=[47.3842, 8.53185], zoom_start=11)
        long = stops_filt[stops_filt['stop_id']== stops_ids[final_result[0]]][['stop_lon']].values.tolist()[0][0]
        lat = stops_filt[stops_filt['stop_id']== stops_ids[final_result[0]]][['stop_lat']].values.tolist()[0][0]
        folium.Marker(location=[lat, long],popup=stops_ids[final_result[0]], tooltip=final_time[0],icon=folium.Icon(color="blue", icon="info-sign"),).add_to(m)
        
        #print(final_result, final_time, final_trips)
        
        for i in range(len(final_result)-1):
            timetable_next = stops_filt[stops_filt['stop_id']== stops_ids[final_result[i]]]
            timetable_this = stops_filt[stops_filt['stop_id']== stops_ids[final_result[i+1]]]
            
            
            long_1 = timetable_next[['stop_lon']]\
            .values.tolist()[0][0]
            lat_1 = timetable_next[['stop_lat']]\
            .values.tolist()[0][0]
            long_2 = timetable_this[['stop_lon']]\
            .values.tolist()[0][0]
            lat_2 = timetable_this[['stop_lat']]\
            .values.tolist()[0][0]
            
            arrival_time_next = final_time[i]
            arrival_time_this = final_time[i+1]
            
            trip_id = final_trips[i]
            print(trip_id)
#             trip_info = {}
#             if trip_id == 'walk':
#                 trip_info = {
#                     'trip_id':'walk',
#                     'route_desc':'walk',
#                     'departure_time' :  arrival_time_this,
#                     'arrival_time': arrival_time_next
#                 }
#             else:
                #arrival_time =  timetable_next[timetable_next['trip_id']== trip_id][['arrival_time']]\
            #.values.tolist()[0][0]
#                 departure_time_this = timetable_this[timetable_this['trip_id']== trip_id][['departure_time']]\
#             .values.tolist()[0][0]
#                 departure_time_next = timetable_next[timetable_next['trip_id']== trip_id][['departure_time']]\
#             .values.tolist()[0][0]
#                 route_desc = timetable[timetable['trip_id']== trip_id][['route_desc']]\
#             .values.tolist()[0][0]
                
#             trip_info = {
#                     'trip_id':trip_id,
#                     'route_desc': route_desc,
#                     'departure_time' :  departure_time_this,
#                     'arrival_time': arrival_time_next}
            if i == len(final_result)-2:
                folium.Marker(location=[lat_2, long_2],popup=stops_ids[final_result[i+1]],\
                          tooltip=final_time[i+1],icon=folium.Icon(color="green", icon="info-sign"),).add_to(m)
            else:
                folium.Marker(location=[lat_2, long_2],popup=stops_ids[final_result[i+1]],\
                          tooltip=final_time[i+1],icon=folium.Icon(color="red", icon="info-sign"),).add_to(m)
#             folium.PolyLine(locations=[(lat_1, long_1),(lat_2, long_2)],popup=trip_info,\
#                             tooltip=trip_info, weight=2,color = 'blue').add_to(m)
            folium.PolyLine(locations=[(lat_1, long_1),(lat_2, long_2)], weight=2,color = 'blue').add_to(m)
            
        display(m)  
Button.on_click(run)


## Discussion:

### Advantages:
1. In our route planing algorithm, we do not need to construct a graph in implementation. Comapre to other graph structure based algorithms(for example Dijkstra, or weighted A*), our algorithmis more efficient compare to theirs. And as shown in the validation procedure, our route planing algorithm get a correct path.
2. Embed the calculation of destination reaching probability into the RAPTOR algorithm. The original RAPTOR algorithm do not contain the calculation of destination reaching probability, because the original RAPTOR algorithm is board first search based, in each round , it do not construct a complete path. However, our's modified version is based deep first search, in which the algorithm construct a complete path in each round ,thus we could compute the success probability of a path in each round, and throw away the invalid one.
3. Estimatioin of delay is unbiased since we use the frequence to estimate the delay probability.

### Shortcomings and possible improvements:
1. Our procedure of estimating the delay probability is not so efficient, since whenever we need to calculate the delay probability, we need to query the delay table and compute the frequence. An alternative approach is to  model the delay's distributions  at each stops, instead of just counting the frequence, which can increase running speed.
2. More effort can be put on UI design. The route given in the map picture is simplified as straight lines in our version. An alternative approach is that we could show each stops through each trips, which looks better. 


## Distribution of work:
- Xiaoqi Ma and Xiaoyu Lin mainly works on  the data preparation and build the predictive model. 
- Zhenyu Zhu and Diyuan Wu mainly works on the route planing algorithm, and Zhenyu Zhu also works on the user interface 
- Zhengqing Wu mainly works on the validation
- All members in our team discuss for the framework of this project
